In [13]:
from bokeh.plotting import figure
import param
import numpy as np
import panel as pn
pn.extension()

class Shape(param.Parameterized):

    radius = param.Number(default=1, bounds=(0, 1))

    def __init__(self, **params):
        super(Shape, self).__init__(**params)
        self.figure = figure(x_range=(-1, 1), y_range=(-1, 1))
        self.renderer = self.figure.line(*self._get_coords())


    def view(self):
        return self.figure


class Circle(Shape):

    n = param.Integer(default=100, precedence=-1)

    def _get_coords(self):
        angles = np.linspace(0, 2 * np.pi, self.n + 1)
        return (self.radius * np.sin(angles),
                self.radius * np.cos(angles))

    @param.depends('radius', watch=True)
    def update(self):
        xs, ys = self._get_coords()
        self.renderer.data_source.data.update({'x': xs, 'y': ys})


class NGon(Circle):

    n = param.Integer(default=3, bounds=(3, 10), precedence=1)

    @param.depends('radius', 'n', watch=True)
    def update(self):
        xs, ys = self._get_coords()
        self.renderer.data_source.data.update({'x': xs, 'y': ys})

In [14]:
shapes = [NGon(), Circle()]

class ShapeViewer(param.Parameterized):
    
    shape = param.ObjectSelector(default=shapes[0], objects=shapes)
    
    @param.depends('shape')
    def view(self):
        return self.shape.view()
    
    @param.depends('shape', 'shape.radius')
    def title(self):
        return '## %s (radius=%.1f)' % (type(self.shape).__name__, self.shape.radius)
    
    def panel(self):
        return pn.Column(self.title, self.view)

In [15]:
viewer = ShapeViewer()

pn.Row(viewer.param, viewer.panel())

Row
    [0] Column(margin=5, name='ShapeViewer', width=300)
        [0] StaticText(value='<b>ShapeViewer</b>')
        [1] Row(width_policy='max')
            [0] Select(margin=(5, 0, 5, 10), name='Shape', options=OrderedDict([('NGon00163',...]), value=NGon, width=240)
            [1] Toggle(align='end', button_type='primary', height_policy='fit', margin=(0, 0, 5, 10), max_height=30, max_width=20, name='⋮')
    [1] Column
        [0] ParamMethod(method)
        [1] ParamMethod(method)